In [1]:
import os
import cv2
import math
import random
import numpy as np
import datetime as dt
import tensorflow as tf
import pathlib 
import json
import imageio 
import tensorflow as tf 
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow_hub as hub
from tqdm import tqdm
from moviepy.editor import  *
from collections import deque
%matplotlib inline

from sklearn.model_selection import train_test_split
# Run imports
from official.vision.configs import video_classification
from official.projects.movinet.configs import movinet as movinet_configs
from official.projects.movinet.modeling import movinet
from official.projects.movinet.modeling import movinet_layers
from official.projects.movinet.modeling import movinet_model
from official.projects.movinet.tools import export_saved_model
from tfrecords import parse_example




from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

In [2]:
tf.test.gpu_device_name()
for gpu in tf.config.experimental.list_physical_devices("GPU"):
    tf.config.experimental.set_virtual_device_configuration(gpu,[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)])
    

In [3]:
ss_files = pathlib.Path('.') # current path

datafile_path = pathlib.Path('./')/'records/tfrecords' #records folder
datafile_path.mkdir(parents=True, exist_ok=True)


datafile_prefix = pathlib.Path('dataset/data')  #path to videos 

# we will save the files in the same place as our datafiles ()
tfrecords_save_path = datafile_path
train_path = tfrecords_save_path/'train/' # path to training shards 
val_path = tfrecords_save_path/'val/' # path to val shards 
test_path = tfrecords_save_path/'test/' # path to test shards 


In [4]:
def get_shards(path):
    shards = [str(path) for path in path.iterdir() if '.tfrecord' == path.suffix]
    return shards

train_shards=get_shards(train_path)
val_shards=get_shards(val_path)
test_shards=get_shards(test_path)


In [5]:

# create dataset with train shards
# customize adding: .map(your_tf_function) after '.map(parse_example)\'
# Note: sharding allows us to shuffle the data efficiently
def get_data(shards):
  dataset = tf.data.TFRecordDataset(shards)
  

  dataset=dataset.map(parse_example)
  #dataset=dataset.repeat(1000)
  dataset=dataset.batch(1)
  data=iter(dataset)
  #data=dataset.as_numpy_iterator()
  return data
#.shuffle(100)

In [6]:
train_dataset=get_data(train_shards)
test_dataset=get_data(test_shards)
val_dataset=get_data(val_shards)


In [ ]:
train_dataset

In [ ]:
tf.compat.v1.data.get_output_types(train_dataset)

(tf.float32, tf.int64)

In [ ]:
tf.compat.v1.data.get_output_shapes(train_dataset)


(TensorShape([None, None, None, None, None]), TensorShape([None]))

In [ ]:
tf.compat.v1.data.get_output_classes(dataset)


(tensorflow.python.framework.ops.Tensor,
 tensorflow.python.framework.ops.Tensor,
 tensorflow.python.framework.ops.Tensor)

# OLD Model

In [ ]:
initial_learning_rate = 0.01
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=20, decay_rate=0.96, staircase=True
)

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    "signlanguage.h5", save_best_only=True
)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    patience=10, restore_best_weights=True
)
def create_model():
    model=Sequential()
    model.add(Input((None,224, 224, 3)))
    #model.add(Flatten())

    model.add(Conv3D(filters = 64, kernel_size = (1,3, 3), activation = 'relu'))
    model.add(Conv3D(filters = 64, kernel_size = (1,3, 3), activation = 'relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling3D(pool_size = (1,2, 2)))
    model.add(GlobalAveragePooling3D())
    model.add(Dense(256, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dense(2, activation = 'softmax'))

    model.summary()

    return model


In [ ]:
initial_learning_rate = 0.01
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=20, decay_rate=0.96, staircase=True
)

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    "signlanguage.h5", save_best_only=True
)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    patience=10, restore_best_weights=True
)
def get_model(width=224, height=224, depth=None):
    """Build a 3D convolutional neural network model."""

    inputs = Input((depth,height, width,  3))

    x = Conv3D(filters=64, kernel_size=3, activation="relu")(inputs)
    x = MaxPool3D(pool_size=2)(x)
    x = BatchNormalization()(x)

    x = Conv3D(filters=64, kernel_size=3, activation="relu")(x)
    x = MaxPool3D(pool_size=2)(x)
    x = BatchNormalization()(x)

    x = Conv3D(filters=128, kernel_size=3, activation="relu")(x)
    x = MaxPool3D(pool_size=2)(x)
    x = BatchNormalization()(x)

    x = Conv3D(filters=256, kernel_size=3, activation="relu")(x)
    x = MaxPool3D(pool_size=2)(x)
    x = BatchNormalization()(x)
    #x=Flatten()(x)

    #x = GlobalAveragePooling3D()(x)
    #x = Dense(units=512, activation="relu")(x)
    #x = Dropout(0.3)(x)
    x=Bidirectional(ConvLSTM2D(512,kernel_size=3))(x)
    

    outputs = Dense(units=2, activation="sigmoid")(x)

    # Define the model.
    model = Model(inputs, outputs, name="3dcnn")
    model.summary()
    return model


# MoVi net


In [7]:

batch_size = 8
num_frames = 8
frame_stride = 10
resolution = 172

In [15]:
model_id = 'a0'

tf.keras.backend.clear_session()

backbone = movinet.Movinet(model_id=model_id)
model = movinet_model.MovinetClassifier(backbone=backbone, num_classes=600)
model.build([1, 1, 1, 1, 3])

# Load pretrained weights
#!wget https://storage.googleapis.com/tf_model_garden/vision/movinet/movinet_a0_base.tar.gz -O movinet_a0_base.tar.gz -q
!tar -xvf movinet_a0_base.tar.gz

checkpoint_dir = 'movinet_a0_base'
checkpoint_path = tf.train.latest_checkpoint(checkpoint_dir)
checkpoint = tf.train.Checkpoint(model=model)
status = checkpoint.restore(checkpoint_path)
status.assert_existing_objects_matched()
model.summary()

Model: "movinet_classifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 image (InputLayer)          [(None, None, None, None  0         
                             , 3)]                               
                                                                 
 movinet (Movinet)           ({'stem': (None, None, N  911583    
                             one, None, 8),                      
                              'block0_layer0': (None,            
                              None, None, None, 8),              
                              'block1_layer0': (None,            
                              None, None, None, 32),             
                              'block1_layer1': (None,            
                              None, None, None, 32),             
                              'block1_layer2': (None,            
                              None, None, None, 

In [12]:
def build_classifier(backbone, num_classes, freeze_backbone=False):
  """Builds a classifier on top of a backbone model."""
  model = movinet_model.MovinetClassifier(
      backbone=backbone,
      num_classes=num_classes)
  model.build([batch_size, num_frames, resolution, resolution, 3])
  if freeze_backbone:
    for layer in model.layers[:-1]:
      layer.trainable = False
    model.layers[-1].trainable = True
  
  
  return model



# Wrap the backbone with a new classifier to create a new classifier head
# with num_classes outputs (101 classes for UCF101).
# Freeze all layers except for the final classifier head.
num_classes=1

#model = build_classifier(backbone, num_classes, freeze_backbone=True)
for l in model.layers[:-1]:
  l.trainable=False

model.layers[-1].trainable=True
model =hub.KerasLayer(model,input_shape=[None,None,None,3])


model=Sequential([model,Dense(50,activation='softmax')])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 600)               3126071   
                                                                 
 dense (Dense)               (None, 50)                30050     
                                                                 
Total params: 3,156,121
Trainable params: 30,050
Non-trainable params: 3,126,071
_________________________________________________________________


TypeError: call() missing 1 required positional argument: 'inputs'

In [13]:
num_epochs = 3

train_steps = 720 // batch_size
total_train_steps = train_steps * num_epochs
test_steps = 450 // batch_size

loss_obj = tf.keras.losses.CategoricalCrossentropy(
    from_logits=True,
    label_smoothing=0.1)

metrics = [
    tf.keras.metrics.TopKCategoricalAccuracy(
        k=1, name='top_1', dtype=tf.float32),
    tf.keras.metrics.TopKCategoricalAccuracy(
        k=5, name='top_5', dtype=tf.float32),
]

initial_learning_rate = 0.01
learning_rate = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate, decay_steps=total_train_steps,
)
optimizer = tf.keras.optimizers.RMSprop(
    learning_rate, rho=0.9, momentum=0.9, epsilon=1.0, clipnorm=1.0)

model.compile(loss=loss_obj, optimizer=optimizer, metrics=metrics)

callbacks = [
    tf.keras.callbacks.TensorBoard(),
]

In [14]:
results = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=num_epochs,
    steps_per_epoch=train_steps,
    validation_steps=test_steps,
    callbacks=callbacks,
    validation_freq=1,
    verbose=1)


Got `type(handle)`: <class 'official.projects.movinet.modeling.movinet_model.MovinetClassifier'>



Got `type(handle)`: <class 'official.projects.movinet.modeling.movinet_model.MovinetClassifier'>


Epoch 1/3


ValueError: in user code:

    File "d:\final final code\env\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "d:\final final code\env\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "d:\final final code\env\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "d:\final final code\env\lib\site-packages\keras\engine\training.py", line 860, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "d:\final final code\env\lib\site-packages\keras\engine\training.py", line 918, in compute_loss
        return self.compiled_loss(
    File "d:\final final code\env\lib\site-packages\keras\engine\compile_utils.py", line 201, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "d:\final

Model: "3dcnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, None, 224, 224,   0         
                             3)]                                 
                                                                 
 conv3d_24 (Conv3D)          (None, None, 222, 222, 6  5248      
                             4)                                  
                                                                 
 max_pooling3d_24 (MaxPoolin  (None, None, 111, 111, 6  0        
 g3D)                        4)                                  
                                                                 
 batch_normalization_24 (Bat  (None, None, 111, 111, 6  256      
 chNormalization)            4)                                  
                                                                 
 conv3d_25 (Conv3D)          (None, None, 109, 109, 6  110656

In [ ]:
model.compile(optimizer='adam',loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])


In [ ]:
history=model.fit(dataset ,epochs=10,batch_size=32 ,steps_per_epoch=100)

In [ ]:
tf.keras.utils.plot_model(model, show_dtype=True, show_shapes=True)

In [ ]:
# Install packages

# tf-models-official is the stable Model Garden package
# tf-models-nightly includes latest changes
!pip install -q tf-models-official

# Install the mediapy package for visualizing images/videos.
# See https://github.com/google/mediapy
#!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
!pip install -q mediapy

# Due to a bug, we reinstall opencv
# See https://stackoverflow.com/q/70537488
!pip uninstall -q -y opencv-python-headless
!pip install -q "opencv-python-headless>4.3"

In [ ]:
!pip numpy --version


ERROR: unknown command "numpy"



In [ ]:
!pip install numpy --upgrade
 

  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
